# Convolutional Neural Network
## Load data
[notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html)

In [31]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

pickle_file = './data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


## Reformate data
- convolutions need the image data formatted as a cube (width by height by channels)
- labels as float 1-hot encodings.

In [32]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


# Define the accuracy function

In [33]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

# Build a convoulutional nueral network

In [34]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.845486
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.965092
Minibatch accuracy: 31.2%
Validation accuracy: 37.6%
Minibatch loss at step 100: 1.023087
Minibatch accuracy: 68.8%
Validation accuracy: 69.3%
Minibatch loss at step 150: 0.649709
Minibatch accuracy: 81.2%
Validation accuracy: 74.8%
Minibatch loss at step 200: 0.846188
Minibatch accuracy: 75.0%
Validation accuracy: 77.9%
Minibatch loss at step 250: 1.247667
Minibatch accuracy: 62.5%
Validation accuracy: 77.0%
Minibatch loss at step 300: 0.375850
Minibatch accuracy: 93.8%
Validation accuracy: 79.3%
Minibatch loss at step 350: 0.596294
Minibatch accuracy: 93.8%
Validation accuracy: 75.6%
Minibatch loss at step 400: 0.278258
Minibatch accuracy: 87.5%
Validation accuracy: 79.6%
Minibatch loss at step 450: 0.863260
Minibatch accuracy: 81.2%
Validation accuracy: 78.6%
Minibatch loss at step 500: 0.636083
Minibatch accuracy: 93.8%
Validation accuracy: 80.7%
Mi

# Improve performance
- More layers
- Dropout
- L2 regularization
- Learning rate decay

In [43]:
# image_size = 28 num_labels = 10 num_channels = 1
# batch_size = 16 patch_size = 5 depth = 16 
num_hidden1 = 512
num_hidden2 = 256
num_hidden3 = 128

reg_weight = 0.001
reg = 0
keep_prob_value = 1 # if the value is 1, no dropout

decay_steps = 500
decay_rate = 0.95
starter_learning_rate = 0.05

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.0, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

#def build_relu_layer(tf_dataset, weights, biases):
#  return  tf.nn.relu(tf.matmul(tf_dataset, weights)+biases)

graph = tf.Graph()
with graph.as_default():
   
  # specify input
  tf_dataset = tf.placeholder(tf.float32, shape = [None, image_size, image_size, num_channels])
  tf_labels = tf.placeholder(tf.float32, shape = [None, num_labels])
  keep_prob = tf.placeholder(tf.float32)
    
  # Convolutional layer 1
  W_conv1 = weight_variable([patch_size, patch_size, num_channels, depth])
  b_conv1 = bias_variable([depth])
  
  h_conv1 = tf.nn.relu(conv2d(tf_dataset, W_conv1) + b_conv1)
  reg += reg_weight * tf.nn.l2_loss(W_conv1)
  
  # Max plooing layer 1
  h_pool1 = max_pool_2x2(h_conv1)
 
  # Convolutional layer 2
  W_conv2 = weight_variable([patch_size, patch_size, depth, depth])
  b_conv2 = bias_variable([depth])
  
  h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
  reg += reg_weight * tf.nn.l2_loss(W_conv2)
  
  # Max plooing layer 2
  h_pool2 = max_pool_2x2(h_conv2)

  # Densely connected layer 1
  shape = h_pool2.get_shape().as_list()

  W_fc1 = weight_variable(
      [shape[1] * shape[2] * shape[3], num_hidden1])
  b_fc1 = bias_variable([num_hidden1])

  h_pool2_flat = tf.reshape(h_pool2, 
                            [-1, shape[1] * shape[2] * shape[3]])
  h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
  reg += reg_weight * tf.nn.l2_loss(W_fc1)
    
  # Add dropout
  h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
  # Densely connected layer 2
  W_fc2 = weight_variable([num_hidden1, num_hidden2])
  b_fc2 = bias_variable([num_hidden2])
  
  h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
  reg += reg_weight * tf.nn.l2_loss(W_fc2)
  
  # Add dropout
  h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

  # Densely connected layer 3
  W_fc3 = weight_variable([num_hidden2, num_hidden3])
  b_fc3 = bias_variable([num_hidden3])
  
  h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)
  reg += reg_weight * tf.nn.l2_loss(W_fc3)
  
  # Add dropout
  h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)
    
  # Readout layer
  W_fc4 = weight_variable([num_hidden3, num_labels])
  b_fc4 = bias_variable([num_labels])

  logits = tf.matmul(h_fc3_drop, W_fc4) + b_fc4
  reg += reg_weight * tf.nn.l2_loss(W_fc4)
   
  # Training computation.
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_labels, logits=logits))
    
  # Add L2 regulization
  loss += reg
  
  # Optimizer, learning rate decay
  # decayed_learning_rate = learning_rate * decay_rate ^ (global_step / decay_steps)
  
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 
                                             decay_steps, decay_rate, staircase = True)
    
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(
      loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  prediction = tf.nn.softmax(logits)

num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_dataset : batch_data, tf_labels : batch_labels, keep_prob : keep_prob_value}
    _, l, train_predictions = session.run(
      [optimizer, loss, prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(train_predictions, batch_labels))
      valid_predictions = prediction.eval(
          {tf_dataset : valid_dataset, tf_labels : valid_labels, keep_prob : 1.0})
      print("Validation accuracy: %.1f%%" % accuracy(valid_predictions, valid_labels))
      test_predictions = prediction.eval(
          {tf_dataset : test_dataset, tf_labels : test_labels, keep_prob : 1.0})
      print("Test accuracy: %.1f%%" % accuracy(test_predictions, test_labels))
  test_predictions = prediction.eval(
       {tf_dataset : test_dataset, tf_labels : test_labels, keep_prob : 1.0})
  print("Test accuracy: %.1f%%" % accuracy(test_predictions, test_labels))

Initialized
Minibatch loss at step 0: 4.629650
Minibatch accuracy: 12.5%
Validation accuracy: 9.1%
Test accuracy: 9.1%
Minibatch loss at step 1000: 2.371767
Minibatch accuracy: 87.5%
Validation accuracy: 84.3%
Test accuracy: 91.1%
Minibatch loss at step 2000: 1.933069
Minibatch accuracy: 100.0%
Validation accuracy: 86.6%
Test accuracy: 93.0%
Minibatch loss at step 3000: 2.646741
Minibatch accuracy: 87.5%
Validation accuracy: 87.3%
Test accuracy: 93.8%
Minibatch loss at step 4000: 1.996800
Minibatch accuracy: 87.5%
Validation accuracy: 88.3%
Test accuracy: 94.7%
Minibatch loss at step 5000: 2.396704
Minibatch accuracy: 81.2%
Validation accuracy: 88.2%
Test accuracy: 94.3%
Minibatch loss at step 6000: 1.462615
Minibatch accuracy: 100.0%
Validation accuracy: 88.8%
Test accuracy: 94.9%
Minibatch loss at step 7000: 1.913931
Minibatch accuracy: 87.5%
Validation accuracy: 89.2%
Test accuracy: 95.0%
Minibatch loss at step 8000: 1.776298
Minibatch accuracy: 81.2%
Validation accuracy: 89.5%
Test